In [ ]:
from pathlib import Path
import shutil

import teehr

In [ ]:
import hvplot.pandas  # noqa
import holoviews as hv
from bokeh.models import DatetimeTickFormatter

plot_width = 800
plot_height = 500
plot_bgcolor = "#F5F5F5"

hv.opts.defaults(
    hv.opts.Curve(width=plot_width, height=plot_height, bgcolor=plot_bgcolor),
    hv.opts.Scatter(width=plot_width, height=plot_height, bgcolor=plot_bgcolor),
    hv.opts.Area(width=plot_width, height=plot_height, bgcolor=plot_bgcolor),
    hv.opts.Points(width=plot_width, height=plot_height, bgcolor=plot_bgcolor),
)
from bokeh.io import output_notebook
output_notebook()

Let's use a pre-created test Evaluation from the HEFS s3 bucket.

Contents:
- 1 USGS location, 5 years of hourly streamflow
- 1 HEFS locations
  - 5 years of hindcasts
  - 1 per day, 6-hour timesteps
  - 66 members each

In [ ]:
TEMP_DIR = Path(Path.home(), "temp", "teehr_playground")
# shutil.rmtree(TEMP_DIR, ignore_errors=True)
# TEMP_DIR.mkdir(parents=True)
# path_str = TEMP_DIR.as_posix()

In [ ]:
%%time
! wget -P "$path_str" https://ciroh-rti-hefs-data.s3.us-east-2.amazonaws.com/teehr/test_evaluations/ckfn6bvr_evaluation.tar.gz
! tar -xzvf "$path_str"/ckfn6bvr_evaluation.tar.gz -C "$path_str"
! rm "$path_str"/ckfn6bvr_evaluation.tar.gz

In [ ]:
ev = teehr.Evaluation(dir_path=Path(TEMP_DIR, "ckfn6bvr_evaluation"))
ev.enable_logging()

In [ ]:
ev.locations.to_geopandas()

In [ ]:
ev.location_crosswalks.to_pandas()

In [ ]:
prim_df = ev.primary_timeseries.filter(
    "configuration_name = 'usgs_observations'"
).to_pandas()

prim_plot = prim_df.hvplot(
    x="value_time",
    y="value",
    xlabel="Date",
    ylabel="Streamflow (cfs)",
    width=plot_width,
    height=plot_height,
    bgcolor=plot_bgcolor,
    line_color="blue",
    line_width=2,
    grid=True,
)
prim_plot

### Skill score

The skill of a model relative to some reference model/timeseries such as climatology can be computed. Methods to create this reference timeseries (or hindcast) are also included.

Here, we'll first compute "climatology" based on the 5-year USGS timeseries, taking the mean of the primary timeseries at each hour of the year. Since we're creating a new timeseries, we also need to add a configuration object.

### Calculate the mean of the USGS observations by hour-of-year, saving as a new primary timeseries ("hourly normals" or "climatology").

We can make use of TEEHR's timeseries generator classes to perform this operation. Hourly normals are considered a "signature" timeseries in TEEHR, since they operate on a single timeseries. 

In [ ]:
from teehr import SignatureTimeseriesGenerators as sts

from teehr.models.filters import TableFilter

# Define the operation to perform.
ts_normals = sts.Normals()
ts_normals.temporal_resolution = "hour_of_year"
ts_normals.summary_statistic = "mean"           # the default

# Define the timeseries to use as input to the operation.
input_ts = TableFilter()
input_ts.table_name = "primary_timeseries"
input_ts.filters = [
    "configuration_name = 'usgs_observations'",
    "unit_name = 'm^3/s'",
    "variable_name = 'streamflow_hourly_inst'"
]

Note. We see some warnings here (and a performance hit) related to window operation which forward and back-fills NaN values. This is because there is only a single partition in this case (defined by: ["location_id", "variable_name", "unit_name", "configuration_name", "reference_time"]). Might need a better approach to this edge case.

In [ ]:
%%time
ev.generate.signature_timeseries(
    method=ts_normals,
    input_table_filter=input_ts,
    start_datetime="2000-01-01T12:00:00",
    end_datetime="2005-01-31T12:00:00",
    timestep="1 hour"
).write()  # default destination: "primary_timeseries"

In [ ]:
clim_df = ev.primary_timeseries.filter(
    "variable_name = 'streamflow_hour_of_year_mean'"
).to_pandas()

clim_plot = clim_df.hvplot(
    x="value_time",
    y="value",
    xlabel="Date",
    ylabel="Streamflow (cfs)",
    width=plot_width,
    height=plot_height,
    bgcolor=plot_bgcolor,
    line_color="magenta",
    line_width=2,
    grid=True,
)
prim_plot * clim_plot

In [ ]:
import calendar

def adjust_hour_of_year(date):
    if calendar.isleap(date.year):
        if date.month == 2 and date.day == 29:
            # Assign to Feb.28 during leap years
            return 58 * 24 + date.hour
        elif date.month > 2:
            return (date.dayofyear - 2) * 24 + date.hour
        else:
            return (date.dayofyear - 1) * 24 + date.hour
    else:
        return (date.dayofyear - 1) * 24 + date.hour

In [ ]:
prim_df["hour_of_year"] = prim_df["value_time"].apply(adjust_hour_of_year)

In [ ]:
prim_df.hour_of_year.min(), prim_df.hour_of_year.max()

In [ ]:
manual_df = prim_df[["value_time", "value", "hour_of_year"]].groupby(["hour_of_year"]).mean()

In [ ]:
manual_df = prim_df.groupby(["hour_of_year"])["value"].mean()

In [ ]:
manual_df

In [ ]:
joined_df = prim_df.join(manual_df, on="hour_of_year", rsuffix="_manual")

In [ ]:
manual_plot = joined_df.hvplot(
    x="value_time",
    y="value_manual",
    xlabel="Date",
    ylabel="Streamflow (cfs)",
    width=plot_width,
    height=plot_height,
    bgcolor=plot_bgcolor,
    line_color="black",
    line_width=1,
    grid=True,
    # alpha=0.75,
)
clim_plot * manual_plot

### Assemble a reference forecast by assigning the climatology values to a HEFS forecast member template

To compare the performance of the HEFS hindcast against a hindcast based on climatology, we can create a "reference forecast" (hindcast in this case) based on the climatology timeseries we just created. We'll assign values from the climatology timeseries to a member of the HEFS hindcast according to the value_time. In this case we also aggregate to 6-hr time steps.

In [ ]:
ev.secondary_timeseries.distinct_values("configuration_name")

Here we'll make use of a Benchmark Forecast Generator class to perform the operation. These create a forecast by assigning values from one timeseries to a template forecast.

In [ ]:
from teehr import BenchmarkForecastGenerators as bm

ref_fcst = bm.ReferenceForecast()

reference_ts = TableFilter(
    table_name="primary_timeseries",
    filters=[
        "configuration_name = 'usgs_observations'",
        "unit_name = 'm^3/s'",
        "variable_name = 'streamflow_hour_of_year_mean'"
    ]
)

template_ts = TableFilter(
    table_name="secondary_timeseries",
    filters=[
        "configuration_name = 'MEFP'",
        "variable_name = 'streamflow_hourly_inst'",
        "unit_name = 'm^3/s'",
        "member = '1993'"
    ]
)

In [ ]:
%%time
ev.configurations.add(
    [
        teehr.Configuration(
            name="reference_climatology_forecast",
            type="secondary",
            description="Reference forecast based on USGS climatology summarized by hour of year"
        )
    ]
)
ev.generate.benchmark_forecast(
    method=ref_fcst,
    reference_timeseries=reference_ts,
    template_timeseries=template_ts,
    output_configuration_name="reference_climatology_forecast"
).write(destination_table="secondary_timeseries")

### Let's explore the data a bit

We'll calculate the ensemble min, max, and mean of the HEFS hindcasts so we can visualize the spread 

In [ ]:
from pyspark.sql.functions import avg, min, max, lit

In [ ]:
hefs_sdf = ev.secondary_timeseries.filter("configuration_name = 'MEFP'").to_sdf()

Calculate mean, min, and max across the HEFS ensemble members.

In [ ]:
stats_sdf = hefs_sdf.groupBy(
    "value_time",
    "reference_time",
    "location_id",
    "unit_name",
    "configuration_name",
    "variable_name"
) \
.agg(
    avg("value").alias("mean_value"),
    max("value").alias("max_value"),
    min("value").alias("min_value")
) \
.withColumn("member", lit("1953"))

In [ ]:
join_sdf = stats_sdf.join(
    hefs_sdf,
    on=[
        "value_time",
        "reference_time",
        "location_id",
        "unit_name",
        "configuration_name",
        "variable_name",
        "member"
    ]
)

In [ ]:
%%time
hefs_stats_df = join_sdf.toPandas()
hefs_stats_df.set_index("value_time", inplace=True)

#### Let's choose a single reference time (forecast) to plot

In [ ]:
example_reference_time = "2002-05-07 12:00:00"

# Ensemble stats (min, max, mean).
hefs_stats_subset_df = hefs_stats_df[hefs_stats_df.reference_time == f"{example_reference_time}"].copy()

# The "reference" forecast assembled from climatology (mean of hour-of-year).
ref_subset_df = ev.secondary_timeseries \
    .filter("configuration_name = 'reference_climatology_forecast'") \
    .filter(f"reference_time = '{example_reference_time}'") \
    .to_pandas()
ref_subset_df.set_index("value_time", inplace=True)

In [ ]:
clim_df = ev.primary_timeseries \
    .query(
        filters=[
            "variable_name = 'streamflow_hour_of_year_mean'",
            "value_time >='2002-05-07 12:00'",
            "value_time <= '2002-06-06 12:00'"
        ]
    ).to_pandas()
clim_df.set_index("value_time", inplace=True)

#### Let's plot an overlay of the HEFS ensemble spread, HEFS ensemble mean, Reference forecast (climatology)

In [ ]:
ref_fcst_plot = ref_subset_df.hvplot.line(
    y="value",
    legend=True,
    line_color="purple",
    label="Reference Forecast from Climatology",
    alpha=0.8
)
clim_plot = clim_df.hvplot.line(
    y="value",
    legend=True,
    line_color="blue",
    label="USGS Hourly Climatology"
)
hefs_mean_plot = hefs_stats_subset_df.hvplot.line(
    y="mean_value",
    # by=["reference_time", "member"],
    legend=True,
    label="Ensemble mean",
    alpha=0.8,
    line_color="black"
)
hefs_shaded_plot = hefs_stats_subset_df.hvplot.area(
    y="min_value",
    y2="max_value",
    color='lightblue',
    alpha=0.5,
    legend=True,
    label="HEFS ensemble spread",
    grid=True
)
hefs_shaded_plot * hefs_mean_plot * clim_plot * ref_fcst_plot.opts(title=f"{example_reference_time} Forecast", ylabel="Discharge (cms)")

### Calculate CRPSS

First we need to re-create the joined timeseries table, so that it includes our new timeseries data.

In [ ]:
%%time
ev.joined_timeseries.create(execute_scripts=True, add_attrs=True)

In [ ]:
ev.joined_timeseries.distinct_values("configuration_name")

In [ ]:
import numpy as np

crps = teehr.ProbabilisticMetrics.CRPS()
crps.summary_func = np.mean
crps.estimator = "pwm"
crps.backend = "numba"
crps.reference_configuration = "reference_climatology_forecast"  # this triggers to skill score calculation

In [ ]:
%%time
include_metrics = [crps]
metrics_df = ev.metrics.query(
    include_metrics=include_metrics,
    group_by=[
        "primary_location_id",
        "configuration_name",
        "season"
    ],
    order_by=["configuration_name"],
).to_pandas()
metrics_df